In [45]:
import numpy as np
import sys
import random as rd
import math

In [46]:
# Create list of cities

instance='Atlanta'
n=int(open('../data/'+instance+'.tsp').readlines()[2][len('DIMENSION: '):])
input=np.loadtxt("../data/"+instance+".tsp",skiprows=5,max_rows=n)

In [47]:
input_xy = []
for i in input:
    input_xy.append((i[1], i[2]))

In [48]:
input_xy

[(33665568.0, -84411070.0),
 (33764940.0, -84371819.0),
 (33770889.0, -84358622.0),
 (34233391.0, -84103278.0),
 (33921088.0, -84337342.0),
 (34087634.0, -84530646.0),
 (33683084.0, -84442307.0),
 (33968483.0, -84204487.0),
 (33809946.0, -84438909.0),
 (34070898.0, -84277636.0),
 (33903368.0, -84376924.0),
 (33836703.0, -84481173.0),
 (33792891.0, -84396721.0),
 (33975761.0, -84265625.0),
 (34002830.0, -84146049.0),
 (34091985.0, -84275997.0),
 (33788510.0, -84369260.0),
 (33872130.0, -84261650.0),
 (34003208.0, -84040883.0),
 (33936017.0, -84388948.0)]

In [49]:
class simanneal(object):
    def __init__(self, points, max_iter):
        
        self.points = points
        self.n = len(points)
        self.T = (self.n)**0.5
        self.alpha = 0.9995
        self.min_T = 10**(-6)
        self.max_iter = max_iter
        self.iter = 1
        self.nodelist = [node for node in range(self.n)]
        self.best_dist = float("Inf")
        self.optimal_solution = None
        self.best_list = []
        
    def randompick(self):
        
        selected_node = rd.choice(self.nodelist)
        sol = [selected_node]
        rem_nodes = set(self.nodelist)
        rem_nodes.remove(selected_node)
        while rem_nodes:
            
            nrst_ngbr = min(rem_nodes, key = lambda x: self.eucdist(selected_node, x))
            rem_nodes.remove(nrst_ngbr)
            sol.append(nrst_ngbr)
            
        dist = self.totdist(sol)
        if dist < self.best_dist:
            self.bist_dist = dist
            self.optimal_solution = sol
            
        self.best_list.append(dist)
        
        return sol, dist
    
    def eucdist(self, node1, node2):
        
        point1, point2 = self.points[node1], self.points[node2]
        return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5
    
    
    def totdist(self, sol):
        
        dist = 0
        for i in range(self.n):
            dist += self.eucdist(sol[i%self.n], sol[(i+1)%self.n])
        return dist
    
    def executeanneal(self):
        
        self.present_sol, self.present_dist = self.randompick() 
        while self.T >= self.min_T and self.iter < self.max_iter:
            
            nextelem = list(self.present_sol)
            a = rd.randint(2, self.n - 1)
            b = rd.randint(0, self.n - a)
            nextelem[b:(a+b)] = reversed(nextelem[b:(a+b)])
            self.criterion(nextelem)
            self.T *= self.alpha
            self.iter += 1
            self.best_list.append(self.present_dist)
            
        print("Solution obtained: ", self.present_dist)
        
    def criterion(self, nextelem):
        
        elem_dist = self.totdist(nextelem)
        if elem_dist < self.present_dist:
            self.present_dist, self.present_sol = elem_dist, nextelem
            if elem_dist < self.best_dist:
                self.best_dist, self.optimal_solution = elem_dist, nextelem
        else:
            if rd.random() < self.isaccept(elem_dist):
                self.present_dist, self.present_sol = elem_dist, nextelem
                
    def isaccept(self, elem_dist):
        
        return math.exp(-abs(elem_dist - self.present_dist)/self.T)

In [50]:
sa = simanneal(input_xy, max_iter = 1000)

In [51]:
sa.executeanneal()

Solution obtained:  2089117.5787391618
